In [ ]:
# importing libraries
import os
import pandas as pd
from sklearn import model_selection
import numpy as np
import shutil
import cv2
import random
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
# data directory
data_directory = 'data\Train_data'

In [ ]:
ls

In [ ]:
def get_file_path_df(folder):
    image_path = []
    image_name = [] 
    
    for root, dirs, files in os.walk(folder):
        for f in files:
            if f.endswith(".jpg"): # checking for jpg extension
                image_name.append(f.split('.')[0])
                image_path.append(os.path.join(root, f)) # appending to image_path list

        df_cols = {'image_name':image_name, 'path': image_path} # creating a dictionary for name and path of image
        df = pd.DataFrame(df_cols) # creating a dataframe 
        
    return df

In [ ]:
# getting absolute path of all images and storing in a dataframe called data_df
data_df = get_file_path_df(data_directory)

# displying random 10 rows
data_df.sample(10)

In [ ]:
data_df.shape

1. Data contains 674 images with potholes
2. 80% of the data is used for training and 20% used for validation

In [ ]:
# splitting the data in to train and validation 
train_df, valid_df = model_selection.train_test_split(data_df, test_size=0.2, random_state=21, shuffle=True)

In [ ]:
# Number of images in training set
print(train_df.shape)

In [ ]:
# Number of images in validation set
print(valid_df.shape)

In [ ]:
# loading the csv file containing bounding box coordinates
bbox_df = pd.read_csv("data\labels.csv")
bbox_df.head()

In [ ]:
def process_bbox(img_df, bbox_df, data_type):
    """
    Function to convert bounding box coordinates into YOLO format
    and also to arrange the images and bounding boxes in specified folders
    
    Parameters:
        1. img_df: data frame containing image name and path
        2. bbox_df: data frame containing bounding box coordinates
        3. data_type: type of data train / valid
    """
    
    # drictories for images and labels
    dirs = ['images/train',
            'images/valid',
            'labels/train',
            'labels/valid',
           ]
    
    # if folder does not exist create them
    for d in dirs:
        if not os.path.exists(d):
            os.makedirs(d)
    
    # loop over each row of image data frame
    for _, row in img_df.iterrows():
        yolo_bbox = []
        
        image_name = row['image_name']
        bbox_df.rename(columns = {'ImageID':'image_name'}, inplace = True)
        bbox_array = bbox_df.loc[:, ['XMin', 'XMax', 'YMin', 'YMax']].values
        
        for bbox in bbox_array:
            x_min = bbox[0]
            y_min = bbox[2]
            x_max = bbox[1]
            y_max = bbox[3]

        
            x_center = (x_min + x_max) / 2 
            y_center = (y_min + y_max) / 2 
        
            w = (x_max - x_min)
            h = (y_max - y_min)
            
            yolo_bbox.append([0, # object class 
                              x_center, # bbox x-center
                              y_center, # bbox y-center
                              w, # bbox width
                              h]) # bbox height
            
        yolo_bbox = np.array(yolo_bbox)
        label_path = f"labels/{data_type}/{image_name}.txt"
        
        # saving txt file containing class label and bbox coordinates
        np.savetxt(label_path, yolo_bbox)
        
        img_source = row['path']
        img_desti = f"images/{data_type}/{image_name}.JPG"
        # moving images to the specific folder
        shutil.move(img_source, img_desti)
    print("Done")

In [ ]:
process_bbox(train_df, bbox_df, 'train')

In [ ]:
process_bbox(valid_df, bbox_df, 'valid')

In [ ]:
# clone the yolov5 github repository
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# install all the requirements
!pip install -r requirements.txt

In [ ]:
%%writefile pothole.yaml
train: ..\images\train
val: ..\images\valid
 
#Classes
nc: 1  # number of classes
names: ['pothole']  # class names

# Training

In [ ]:
!python yolov5\train.py --img 640 --batch 12 --epochs 20 --data pothole.yaml --weights yolov5s.pt --cache

# Testing

In [ ]:
!python yolov5\detect.py --save-txt --save-conf --source data\Test_data --weights yolov5\runs\train\exp7\weights\best.pt --conf 0.5